In [2]:
import pandas as pd
import requests 
import os
from dotenv import load_dotenv
from supabase import create_supabase_client, Client

from Functions.ExtractTextFromHTML import ExtractTextFromHTML
from Functions.GetLinksFromWebsite import GetLinksFromWebsite
from Functions.GetRobots_url import GetRobots_url
from Functions.MatchLinksWithCertainKeywords import MatchLinksWithCertainKeywords
from Functions.IsSocialMediaURL import IsSocialMediaURL
from Functions.IsUrlDisallowed import IsUrlDisallowed
from Functions.IsRobotsValid import IsRobotsValid
from Functions.RemovePrimaryLinkFromList import RemovePrimaryLinkFromList
from Functions.RemoveUnnecessaryHtmlElements import RemoveUnnecessaryHtmlElements
from Functions.StripSubpaths import StripSubpaths
from Functions.MatchLinksWithAllKeywords import MatchLinksWithAllKeywords
from Functions.FilterLines import FilterLines


ImportError: cannot import name 'create_supabase_client' from 'supabase' (/Users/mathiesklerck/miniconda3/envs/datascience/lib/python3.11/site-packages/supabase/__init__.py)

### Daten fetchen

In [4]:
#! HIER FEHLT EIN INDEX, DER ALLE KETTEN-UNTERNEHMEN ETC HERAUSFILTERT

# DataFrame für brauchbare Unternehmen anlegen
validCompaniesDF = pd.DataFrame(columns=[
    'ID',
    'WebsiteURL',
    'DiscountText'
])

# DataFrame für unvollständige und nicht brauchbare Unternehmen anlegen
incompleteCompaniesDF = pd.DataFrame(columns=[
    'ID',
    'WebsiteURL',
    'Error'
])

# DataFrame für unvollständige und nicht brauchbare Unternehmen anlegen
bannedScrapingWebsitesDF = pd.DataFrame(columns=incompleteCompaniesDF.keys())

# Error-Websites-DataFrame 
errorWebsitesDF = pd.DataFrame(columns=incompleteCompaniesDF.keys())

# DataFrame für Websites, wo es keine brauchbaren Links gibt
bannedScrapingWebsitesDF = pd.DataFrame(columns=incompleteCompaniesDF.keys())

# DataFrame für Websites, wo es keine brauchbaren Links gibt
noMatchingLinksDF = pd.DataFrame(columns=incompleteCompaniesDF.keys())

# DataFrame für Websites, deren Website-URL zu Facebook führt
socialMediaWebsitesDF = pd.DataFrame(columns=incompleteCompaniesDF.keys())

# Scraping-Header setzen
header = {
    'User-Agent': 'DiscountableScraper/1.0 (Purpose: Collecting data on disability discounts to present them in an app for users; Contact: mathies.klerck@stud.hs-flensburg.de)'
}

# Durch die Unternehmen iterieren
for index, row in fetchedLocalitiesDF.iterrows():
    id = None
    websiteURL = None

    robotsExists = True

    # Places setzen
    id = row['locality_id']

    # Website setzen 
    websiteURL = row['website']
    
    # Wenn Facebook, dann aussortieren
    if IsSocialMediaURL(websiteURL):
        print(f"Link führt zu Social-Media: {websiteURL}")
        socialMediaWebsitesDF.loc[len(socialMediaWebsitesDF)] = [id, websiteURL, "SM"]
        continue

    # Die URL zuschneiden
    websiteURL = StripSubpaths(websiteURL)

    # Robots.txt fetchen und Status überprüfen
    robots_url = GetRobots_url(websiteURL)
    try:
        robotsResponse = requests.get(robots_url, headers=header)
    except:
        print(f"Beim Website-Aufruf gab einen Error: {websiteURL}")
        errorWebsitesDF.loc[len(errorWebsitesDF)] = [id, websiteURL, "c;W"]
        continue

    if robotsResponse.status_code != 200 or IsRobotsValid(robotsResponse.text) == False:
        robotsExists = False

    # Überpüfen, ob die Mainpage bzw. die gesamte Seite nicht gecrawlt werden darf
    if robotsExists == True and IsUrlDisallowed(websiteURL, robotsResponse.text) == True:
        print(f"Scraping der Mainpage nicht erlaubt: {websiteURL}")
        bannedScrapingWebsitesDF.loc[len(bannedScrapingWebsitesDF)] = [id, websiteURL, "d;M"]
        continue
    
    # Main-Page fetchen und Status überprüfen
    mainpageResponse = requests.get(websiteURL, headers=header)
    
    if mainpageResponse.status_code != 200:
        print(f"Fehler beim Abrufen folgender Main-Page der Website: {websiteURL} Status-Code: {mainpageResponse.status_code}")
        errorWebsitesDF.loc[len(errorWebsitesDF)] = [id, websiteURL, f"{robotsResponse.status_code};M"]
        continue

    # Alle Links aus der Main-Page extrahieren
    linksFromMainpage = GetLinksFromWebsite(mainpageResponse.text)

    # Links mit gefundenen Keywords extrahieren
    matchedLinkMainPage = MatchLinksWithAllKeywords(websiteURL, linksFromMainpage, websiteURL)   

    # Den Link speichern, falls existent
    if matchedLinkMainPage == None:
        content = RemoveUnnecessaryHtmlElements(mainpageResponse.text)
        content = ExtractTextFromHTML(content)
        content = FilterLines(content)

        print(f"Fehlende Links auf Mainpage der Website: {websiteURL} Text der Mainpage wurde gespeichert.")
        validCompaniesDF.loc[len(validCompaniesDF)] = [id, websiteURL, content]
        continue

    # Überprüfen, ob der Link gecrawlt werden darf
    pricingPageLink = websiteURL + "/" + matchedLinkMainPage
    if robotsExists == True and IsUrlDisallowed(pricingPageLink, robotsResponse.text) == True:
        print(f"Scraping der Preisseite nicht erlaubt: {pricingPageLink}")
        bannedScrapingWebsitesDF.loc[len(bannedScrapingWebsitesDF)] = [id, websiteURL, "d;P"]
        continue

    # RawText des Preis-Seite fetchen und Status überprüfen
    pricingPageResponse = requests.get(pricingPageLink, headers=header)

    if pricingPageResponse.status_code != 200:
        print(f"Fehler beim Abrufen der Preis-Seite folgender Website: {websiteURL} Status-Code: {pricingPageResponse.status_code}")
        errorWebsitesDF.loc[len(errorWebsitesDF)] = [id, websiteURL, f"{pricingPageResponse.status_code};P"]
        continue

    # RawText von Preis-Seite von HTML-Code und irrelevantem Content bereinigen
    pricingPageContent = RemoveUnnecessaryHtmlElements(pricingPageResponse.text)
    pricingPageContent = ExtractTextFromHTML(pricingPageContent)
    pricingPageContent = FilterLines(pricingPageContent)

    # Alle Links aus der Preis-Seite extrahieren
    linksFromPricingPage = GetLinksFromWebsite(pricingPageResponse.text)

    # Alle Links, die gleich dem von der Mainpage sind, entfernen
    linksFromPricingPage = RemovePrimaryLinkFromList(matchedLinkMainPage, linksFromPricingPage, websiteURL)

    # Link mit passendem Keywords extrahieren
    matchedLinkPricingPage = MatchLinksWithCertainKeywords(websiteURL, linksFromPricingPage, pricingPageLink)

    subPageExists = False
    if matchedLinkPricingPage != None:

        # RawText der Unterseite der Pricing-Page fetchen und Status überprüfen
        pricingSubPageLink = websiteURL + "/" + matchedLinkPricingPage
        pricingSubPageResponse = requests.get(pricingSubPageLink, headers=header)

        if pricingSubPageResponse.status_code != 200:
            print(f"Fehler beim Abrufen der Unterseite der Pricing-Page folgender Website: {websiteURL} Status-Code: {pricingSubPageResponse.status_code}")
            errorWebsitesDF.loc[len(errorWebsitesDF)] = [id, websiteURL, f"{pricingSubPageResponse.status_code};S"]

        else: 
            # RawText von Unterseite von HTML-Code und irrelevantem Content bereinigen
            pricingSubPageContent = RemoveUnnecessaryHtmlElements(pricingSubPageResponse.text)
            pricingSubPageContent = ExtractTextFromHTML(pricingSubPageContent)
            pricingSubPageContent = FilterLines(pricingSubPageContent)
            subPageExists = True

    if subPageExists == True:
        content = f"{pricingPageContent} \n {pricingSubPageContent}"
    else:
        content = pricingPageContent 

    validCompaniesDF.loc[len(validCompaniesDF)] = [id, websiteURL, content]

    print("Content gefunden")

Link führt zu Social-Media: https://www.facebook.com/jayzzfl
Fehlende Links auf Mainpage der Website: http://study-lounge.de Text der Mainpage wurde gespeichert.
Fehlende Links auf Mainpage der Website: https://sadhu-flensburg.de Text der Mainpage wurde gespeichert.
Link führt zu Social-Media: https://www.instagram.com/caveflensburg/
Fehlende Links auf Mainpage der Website: http://www.poppx.de Text der Mainpage wurde gespeichert.
Scraping der Mainpage nicht erlaubt: https://m.facebook.com
Fehlende Links auf Mainpage der Website: http://www.punschwald.de Text der Mainpage wurde gespeichert.
Fehlende Links auf Mainpage der Website: https://heimathafen-seewarte.de Text der Mainpage wurde gespeichert.
Fehlende Links auf Mainpage der Website: http://www.historischer-krug.de Text der Mainpage wurde gespeichert.
Content gefunden


KeyboardInterrupt: 

Jetzt bestellen bei Sadhu Flensburg | Flensburg
Natürlich bieten wir neben Barzahlung auch verschiedene bargeldlose Zahlungsmöglichkeiten an, sodass du immer flexibel bist. Lass dich von uns aus Flensburg nach Indien entführen und tauche ein in ein neues Geschmackserlebnis.Die indische Küche ist bunt, pikant und voller frischer Zutaten, die vereint werden zu leckeren, würzigen und scharfen Gerichten. Bestelle für einen gemütlichen Abend zuhause mit Familie oder Freunden in Flensburg bei deinem Profi für indische Küche: Sadhu. Egal ob Curry, Tandoori oder Biryani: hier findet jeder sein Lieblingsgericht der indischen Küche mit variablem Schärfegrad. Durch beste Zutaten und gekonnte Zubereitung erschaffen wir bei Sadhu in Flensburg indische Gerichte in top Qualität und liefern diese zu dir nach Hause.


# Test-Notebook

In [1]:

from Functionsdsds import RemoveUnnecessaryHtmlElements
from Functionsdsds import ExtractTextFromHTML
from Functionsdsds import FilterLines

with open('rawText.txt', 'r') as file:
    raw = file.read()
raw = RemoveUnnecessaryHtmlElements(raw)
raw = ExtractTextFromHTML(raw)

# Text mit der Filterfunktion reinigen
cleaned_text = FilterLines(raw)
print(cleaned_text)


# print(links)
# match = MatchLinksWithAllKeywords(websiteURL, links, url)
# #! Bei über 
# print(match)
# print(ExtractTextFromHTML(raw))

Unsere Eintrittspreise - PHÄNOMENTA Flensburg
- Der Zutritt für Hunde ist nicht gestattet. Ausnahme: Blinden- oder medizinische Assistenzhunde.
- Die Ausstellung ist zum größten Teil barrierefrei. Ausschließlich zwei kleine Emporen sind über Treppenaufgänge zu erreichen.
Kinder von 3 bis 6 3 Euro / 24 DKK
Für Jugendliche bis 16, Schüler, Studierende, Auszubildende, Schwerbehinderte (70%), Sozialpassinhaber 10 Euro / 80 DKK
14 Euro / 112 DKK
2 Erwachsene + 2 Kinder bis 16 Jahre 40 Euro / 320 DKK
Gruppen (ab 15 Personen)
Begleitpersonen von Ermäßigtengruppen: bis 19 Personen hat eine Begleitperson freien Eintritt (ab 20 Personen zwei Begleitpersonen, ab 30 Personen drei Begleitpersonen, usw.).
Eintritt
AUSSTELLUNG
MINIPHÄNOMENTA
GRUPPENBUCHUNGEN
